In [89]:
!pip install libcst

In [90]:
from libcst import parse_module, Module, Expr, Pass, Comment, CSTTransformer, Comparison, CSTNode, ComparisonTarget
from libcst import matchers
from typing import List, Dict, Union, Tuple, Type
import re
from libcst import Equal, GreaterThanEqual
from libcst import matchers as m

## REFERENCES

- [Nodes](https://libcst.readthedocs.io/en/latest/nodes.html#libcst-nodes)

In [91]:
# Use difflib to show the changes to verify type annotations were added as expected.
import difflib

def printdiff(vanilla, tainted):
    return (
        "".join(
            difflib.unified_diff(vanilla.splitlines(1), tainted.splitlines(1))
        )
    )

In [92]:
import libcst as cst


def gen_transfomers(op1, op2):
    class IsOperatorProvider(cst.BatchableMetadataProvider[Dict]):
    
        @m.call_if_inside(m.BinaryOperation(operator=[getattr(m, op1.__name__)()]))
        def visit_Comparison(self, node: cst.Param) -> None:
            # Mark the node as an equal comparison node
            self.set_metadata(node, dict(comparison=node.comparisons[0], 
                                    original=node.comparisons[0].operator.__class__))
    
    class ApplyTransformer(cst.CSTTransformer):
        METADATA_DEPENDENCIES = (IsOperatorProvider, )
    
        def leave_Comparison(self, vanilla:cst.Name, tainted: cst.Name) -> None:
            meta = self.get_metadata(IsOperatorProvider, vanilla)
            if meta:
                updated_comparison = meta['comparison'].with_changes(operator=op2()) # OP2
                tainted = vanilla.with_changes(comparisons=[updated_comparison])
            return tainted
        def __repr__(self):
            return super().__repr__ + ':' + op1.__name__ +':' + op2.__name__
    
    class ReverseTransformer(cst.CSTTransformer):
        METADATA_DEPENDENCIES = (IsOperatorProvider, )

        def leave_Comparison(self, tainted:cst.Name, vanilla: cst.Name) -> None:
            meta = self.get_metadata(IsOperatorProvider, tainted)
            if meta:
                updated_comparison = meta['comparison'].with_changes(operator=meta['original']())
                vanilla = tainted.with_changes(comparisons=[updated_comparison])
            return vanilla

    ReverseTransformer.__class__.__str__ = lambda self: ReverseTransformer.__str__ + ':' + op1.__name__ +':' + op2.__name__

    return ApplyTransformer, ReverseTransformer    

In [93]:
import itertools
itertools.chain([1,2,3], [4,5,6])

In [94]:
def composite_factory(mutators: List[Transform]):
    class ApplyCompositeTransformer(cst.CSTTransformer):
        METADATA_DEPENDENCIES = itertools.chain(m[0].METADATA_DEPENDENCIES for m in mutators)
        
        def __init__(self):
            super().__init__()
            self._mutators = mutators

        def leave_Comparison(self, vanilla:cst.Name, tainted: cst.Name) -> None:
            for apply, _ in self._mutators:
                vanilla = tainted = apply().leave_Comparison(vanilla, tainted) 
                
            return tainted

    return ApplyCompositeTransformer

In [95]:
from libcst import (Equal, GreaterThanEqual, LessThan, GreaterThan, 
                    LessThanEqual, NotEqual, NotIn, In, Is, IsNot, Not, And, Or, Match)
import collections

str2op = dict([
    ('==', Equal),
    ('>=', GreaterThanEqual),
    ('>', GreaterThan),
    ('<', LessThan),
    ('=<', LessThanEqual),
    ('!=', NotEqual),
    ('not in', NotIn),
    ('in', In),
    ('is', Is),
    ('is not', IsNot),
    ('not', Not),
    ('and', And),
    ('or', Or),
    ('or', Or),
    # ('match', Match),
])

op2str = {v:k for k,v in str2op.items()}

Transform = collections.namedtuple('Transform', ['apply', 'reverse'])
transforms = collections.defaultdict(dict)

for op1 in [ Equal , GreaterThanEqual, LessThan, GreaterThan, LessThanEqual, NotEqual, NotIn, In, Is, IsNot, Not, And, Or,]:
    for op2 in [ Equal, GreaterThanEqual, LessThan, GreaterThan, LessThanEqual, NotEqual, NotIn, In, Is, IsNot, Not, And, Or,]:
        if op1 is op2: continue
        apply, reverse = gen_transfomers(op1, op2)
        transforms[op2str[op1]][op2str[op2]] = Transform(apply, reverse)


In [96]:
len(transforms)

13

In [97]:
import random
random.choice(transforms)

{}

In [100]:
sequence_of_transforms_to_apply = (
    "x >= z and x == y" ,
    [transforms['==']['>='],
     transforms['and']['or']],
    "x >= z or x >= y" ,
)

# pre_wrapper = cst.MetadataWrapper(module)
# change_equal_to_op1, _ = gen_transfomers(Equal, op1)
# new_code = pre_wrapper.visit(change_equal_to_op1)
# print(new_code.code)
# print(op1.__name__, '--->', op2.__name__)
# print('  :', code, )
# print('->:',bugged.code, )
# vanilla = wrapper.visit(reverse())
# print('<-:', vanilla.code)
# print( '---')

code, TLIST, result =  sequence_of_transforms_to_apply

module = cst.parse_module(code)
wrapper = cst.MetadataWrapper(module)
apply = composite_factory(TLIST)
bugged = wrapper.visit(apply())
apply

((<class '__main__.gen_transfomers.<locals>.IsOperatorProvider'>,), (<class '__main__.gen_transfomers.<locals>.IsOperatorProvider'>,))


AttributeError: 'tuple' object has no attribute 'METADATA_DEPENDENCIES'

In [99]:
# printdiff(cst.parse_expression("1 == 2").code, cst.parse_express('1 >= 2').code)